In [1]:
# Imports
import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms # Transformations we can perform on our dataset
import torchvision
from torch.utils.data import DataLoader

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available () else 'cpu')
device

device(type='cpu')

In [3]:
# Hyperparameters
in_channel = 3
num_classes = 10
learning_rate = 1e-3
batch_size = 32
num_epochs = 1

In [10]:
#it comes in datasetloader.py file

import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import io

class CatsAndDogsDataset (Dataset):

  def __init__(self, csv_file, root_dir, transform=None) :

    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

  def __len__ (self):

    return len(self.annotations)

  def __getitem__(self, index):

    img_path = os.path. join(self.root_dir, self.annotations.iloc[index, 0])
    image = io.read_image (img_path)
    y_label = torch. tensor(int (self .annotations.iloc[index, 1]))

    if self.transform:
      image = self.transform(image)

    return (image, y_label)

In [12]:
#from dataloader import CatsAndDogsDataset

# Load Data
dataset = CatsAndDogsDataset(csv_file = '/Users/daniyalkhan/Documents/AI/DL/Pytorch/data/Persian_Car_Plates_YOLOV8/car_number.csv', root_dir = "", transform = transforms.ToTensor ())
train_set, test_set = torch.utils. data.random_split (dataset, [200, 19])

train_loader = DataLoader (dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader (dataset=test_set, batch_size=batch_size, shuffle=True)

In [14]:
# Model
model = torchvision.models. googlenet (pretrained=True)
model.to(device)

/Users/daniyalkhan/anaconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/daniyalkhan/anaconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /Users/daniyalkhan/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100.0%


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [15]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss ()
optimizer = optim.Adam (model.parameters (), lr=learning_rate)

In [ ]:
# Train Network
for epoch in range (num_epochs) :
  losses = []
  for batch_idx, (data, targets) in enumerate (train_loader):
  # Get data to cuda if possible
    data = data. to (device=device)
    targets = targets. to(device=device)
    # forward
    scores = model (data)
    loss = criterion(scores, targets)
    losses.append(loss. item ())
    # backward
    optimizer.zero_grad()
    loss.backward()
    # gradient descent or adam step
    optimizer.step ()
  print(f'Cost at epoch {epoch} is {sum(losses)/len(losses)}' )